<h1> Mass Image Rectification </h1>

In [ ]:
import glob
import json
import os
from multiprocessing import Pool, pool

import cv2
import matplotlib.pyplot as plt
import numpy as np
import csv

from scipy.io import loadmat
import time

<h1> Load parameters file </h1>

In [ ]:
def load_params(params_file):
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)
    
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps

In [ ]:
params_file = '/app/data/stereo_parameters.json'
left_maps, right_maps = load_params(params_file)

<h1> Load all raw stereo image pairs to be rectified </h1>

In [ ]:
def remap_and_save(img_path, output_dir, left_maps, right_maps):
    
    new_path = os.path.join(output_dir, os.path.basename(img_path))    
    img = cv2.imread(img_path)
    if 'left' in img_path:
        img_remap = cv2.remap(img, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    elif 'right' in img_path:
        img_remap = cv2.remap(img, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
    else:
        raise Exception('Image path does not contain side')
    if not os.path.isdir(os.path.dirname(new_path)):
        os.makedirs(os.path.dirname(new_path))
    if not os.path.isfile(new_path):
        cv2.imwrite(new_path, img_remap)






In [ ]:
# specify the directory from which raw stereo frame pairs will be ingested
left_images_directory = '/app/data/images_02/filtered_left_calibration_images'
right_images_directory = '/app/data/images_02/filtered_right_calibration_images'

# define output directory to which all rectified images will be written
rectified_left_images_directory = '/app/data/images_02/rectified_filtered_left_calibration_images'
rectified_right_images_directory = '/app/data/images_02/rectified_filtered_right_calibration_images'

# get list of all stereo frame pairs
left_images = glob.glob('{}/*.jpg'.format(left_images_directory))
right_images = glob.glob('{}/*.jpg'.format(right_images_directory))
stereo_frame_pairs = list(zip(sorted(left_images), sorted(right_images)))

# create list of arguments to be mapped to multiprocessor
arguments_list = []
for sfp in stereo_frame_pairs:
    left_image_path = sfp[0]
    
    left_arguments = [left_image_path, rectified_left_images_directory, left_maps, right_maps]
    arguments_list.append(left_arguments)
    
    right_image_path = sfp[1]
    
    right_arguments = [right_image_path, rectified_right_images_directory, left_maps, right_maps]
    arguments_list.append(right_arguments)


<h1> Run rectification on a per-frame basis on a single process </h1>

<h1> Run rectification en masse </h1>

In [ ]:
# run multiple processes to rectify the images en masse (this may take a while depending on the number of images)

p = Pool(processes=20)
results = p.starmap(remap_and_save, arguments_list[:10])

In [ ]:
starting_time = time.time()
image_counter = 0
for arguments in arguments_list:
    print('here')
    remap_and_save(*arguments)
    image_counter += 1
    if image_counter % 10 == 0:
        time_taken_s = time.time() - starting_time
        fps = image_counter / time_taken_s
        print('Average FPS over {0} images: {1}'.format(image_counter, fps))
    